## 语法错误

又称解析错误
- 语句或表达式在语法上出现错误
- 解析器会输出语法错误的那一行，并显示一个箭头，指向在该行检测到的第一个错误
    - 文件名和行号也被输出
- 错误由箭头指示的位置 *上面* 的 token 引起
    - 至少是在这里被检测出来的

In [1]:
# 缺失冒号
while True print('Hello world')

SyntaxError: invalid syntax (<ipython-input-1-3fc35091614a>, line 2)

## 异常

在执行时检测到的错误
- 不一定导致严重后果
- 大多数异常不会被程序处理

错误信息
- 最后一行显示什么类型的错误
    - [内置异常](https://docs.python.org/zh-cn/3/library/exceptions.html#built-in-exceptions)
        - [ZeroDivisionError](https://docs.python.org/zh-cn/3/library/exceptions.html#ZeroDivisionError)
        - [NameError](https://docs.python.org/zh-cn/3/library/exceptions.html#NameError)
        - [TypeError](https://docs.python.org/zh-cn/3/library/exceptions.html#TypeError)
        - [KeyboardInterrupt](https://docs.python.org/zh-cn/3/library/exceptions.html#KeyboardInterrupt)
        - 标准的异常类型是内置的标识符（而不是保留关键字）
    - 最后一行的剩余部分根据异常类型及其原因提供详细信息
- 以堆栈回溯的形式显示发生异常时的上下文
    - 包含列出源代码行的堆栈回溯
    - 不会显示从标准输入中读取的行

In [2]:
# 除零异常（取余，模数为 0）
10 * (1/0)

ZeroDivisionError: division by zero

In [3]:
# 局部或全局名称未找到 
4 + spam*3

NameError: name 'spam' is not defined

In [4]:
# 类型不匹配
'2' + 2

TypeError: can only concatenate str (not "int") to str

## 处理异常

[try](https://docs.python.org/zh-cn/3/reference/compound_stmts.html#try) 语句的工作原理
- 首先，执行 try 子句 （try 和 except 关键字之间的（多行）语句）；
- 如果没有异常发生，则跳过 *except 子句* 并完成 try 语句的执行；
- 如果在执行 try 子句时发生了异常，则跳过该子句中剩下的部分；
    - 如果异常的类型和 except 关键字后面的异常匹配，则执行 except 子句；然后继续执行 try 语句之后的代码；
    - 如果发生的异常和 except 子句中指定的异常不匹配，则将其传递到外部的 try 语句中；
    - 如果没有找到处理程序，则它是一个 *未处理异常*，执行将停止并显示异常消息。

一个 try 语句可以包含多个 expect 子句
- 指定不同的异常处理程序
- 最多执行一个处理程序

处理程序只处理相应的 try 子句中发生的异常
- 不处理同一 try 语句内其他处理程序中的异常

expect 子句
- 可以将多个异常命名为带括号的元组
- 可以省略异常名
    - 用作通配符
- 可用于打印错误消息
    - 然后重新引发异常，或处理异常
- 可以在异常名称后面指定一个变量
    - 和一个异常实例绑定，参数存储在 `instance.args` 中
        - 异常实例定义了 [\_\_str\_\_](https://docs.python.org/zh-cn/3/reference/datamodel.html#object.__str__)，可以直接打印而无需引用 `.args`
    - 可以在抛出之前实例化异常
        - 根据需要添加属性
    
`try ... except` 语句有一个可选的 else 子句
- 使用时必须放在所有的 except 子句后面
- 在 try 子句不引发异常时执行

In [5]:
# 直到输入有效整数
while True:
    try:
        x = int(input("Please enter a number: "))
        break
    except ValueError:    # 类型不匹配
        print("Oops!  That was no valid number.  Try again...")

Please enter a number: a
Oops!  That was no valid number.  Try again...
Please enter a number: 1


In [6]:
# 将多个异常命名为带括号的元组
try:
    pass
except (RuntimeError, TypeError, NameError):
    pass

如果发生的异常和 except 子句中的类是同一个类或者是它的基类，则异常和 except 子句中的类是兼容的。

但反过来则不成立，列出派生类的 except 子句与基类不兼容。

In [7]:
class B(Exception):
    pass

class C(B):
    pass

class D(C):
    pass

# 依次打印 B，C，D
for cls in [B, C, D]:
    try:
        raise cls()    # 跑抛出异常
    except D:
        print("D")     # 派生类
    except C:
        print("C")     # 派生类
    except B:
        print("B")     # 基类

B
C
D


In [8]:
# 颠倒 expect 子句
for cls in [B, C, D]:
    try:
        raise cls()
    except B:
        print("B")     # 基类
    except D:
        print("D")     # 派生类
    except C:
        print("C")     # 派生类

B
B
B


In [9]:
# else 子句可以避免意外捕获由 try ... except 语句保护的代码未引发的异常
for arg in sys.argv[1:]:
    try:
        f = open(arg, 'r')
    except OSError:
        print('cannot open', arg)
    else:
        print(arg, 'has', len(f.readlines()), 'lines')
        f.close()

NameError: name 'sys' is not defined

发生异常时，可能具有关联值
- 也称为异常参数
- 作为未处理异常的消息的最后一部分打印

参数的存在和类型取决于异常类型

In [10]:
try:
    raise Exception('spam', 'eggs')
except Exception as inst:
    print(type(inst))    # the exception instance
    print(inst.args)     # arguments stored in .args
    print(inst)          # __str__ allows args to be printed directly,
                         # but may be overridden in exception subclasses
    x, y = inst.args     # unpack args
    print('x =', x)
    print('y =', y)

<class 'Exception'>
('spam', 'eggs')
('spam', 'eggs')
x = spam
y = eggs


异常处理程序不仅处理 try 子句中遇到的异常，还处理 try 子句中调用（即使是间接地）的函数内部发生的异常。

In [11]:
def this_fails():
    x = 1/0

try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)

Handling run-time error: division by zero


## 抛出异常

[raise](https://docs.python.org/zh-cn/3/reference/simple_stmts.html#raise) 语句允许强制引发指定的异常
- 参数：要抛出的异常
    - 异常实例
    - 异常类
        - 派生自 [Exception](https://docs.python.org/zh-cn/3/library/exceptions.html#Exception) 的类
        - 通过调用没有参数的构造函数来隐式实例化

In [12]:
# 异常实例
raise NameError('HiThere')

NameError: HiThere

In [13]:
# 异常类
raise ValueError  # shorthand for 'raise ValueError()'

ValueError: 

In [14]:
# 确定是否引发了异常，但不打算处理
try:
    raise NameError('HiThere')
except NameError:
    print('An exception flew by!')
    raise    # 重新引发异常

An exception flew by!


NameError: HiThere

## 用户自定义异常

可以通过创建新的异常类来命名新的异常
- 直接或间接地从 Exception 类派生

定义异常类，它可以执行任何其他类可以执行的任何操作，但通常保持简单，只提供许多属性，这些属性允许处理程序为异常提供有关错误的信息。

在创建可能引发多个不同错误的模块时，通常的做法是为该模块定义的异常创建基类，并为不同错误条件创建特定异常类的子类。

大多数异常都定义为以 Error 结尾的名称，类似于标准异常的命名。

许多标准模块定义了它们自己的异常，以报告它们定义的函数中可能出现的错误。

In [15]:
class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class InputError(Error):
    """Exception raised for errors in the input.

    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """

    def __init__(self, expression, message):
        self.expression = expression
        self.message = message

class TransitionError(Error):
    """Raised when an operation attempts a state transition that's not
    allowed.

    Attributes:
        previous -- state at beginning of transition
        next -- attempted new state
        message -- explanation of why the specific transition is not allowed
    """

    def __init__(self, previous, next, message):
        self.previous = previous
        self.next = next
        self.message = message

## 清理操作

try 语句有另一个可选子句（[finally](https://docs.python.org/zh-cn/3/reference/compound_stmts.html#finally)）
- 用于定义在所有情况下都必须执行的清理操作
- 将作为 try 语句结束前的最后一项任务被执行
    - 不论 try 语句是否产生了异常都会被执行

当异常发生时的一些更复杂的情况：
- 如果在执行 try 子句期间发生了异常，该异常可由一个 except 子句进行处理。 如果异常没有被某个 except 子句所处理，则该异常会在 finally 子句执行之后被重新引发。
- 异常也可能在 except 或 else 子句执行期间发生。 同样地，该异常会在 finally 子句执行之后被重新引发。
- 如果在执行 try 语句时遇到一个 break, continue 或 return 语句，则 finally 子句将在执行 break, continue 或 return 语句之前被执行。
- 如果 finally 子句中包含一个 return 语句，则返回值将来自 finally 子句的某个 return 语句的返回值，而非来自 try 子句的 return 语句的返回值

在实际应用程序中，finally 子句对于释放外部资源（例如文件或者网络连接）非常有用，无论是否成功使用资源。

In [16]:
try:
    raise KeyboardInterrupt
finally:
    print('Goodbye, world!')

Goodbye, world!


KeyboardInterrupt: 

In [17]:
def bool_return():
    try:
        return True
    finally:
        return False

bool_return()

False

finally 子句在任何情况下都会被执行

两个字符串相除所引发的 TypeError 不会由 except 子句处理
- 因此会在 finally 子句执行后被重新引发

In [18]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

# 正确调用
divide(2, 1)

result is 2.0
executing finally clause


In [19]:
# 除零异常
divide(2, 0)

division by zero!
executing finally clause


In [20]:
# 类型不匹配
divide("2", "1")

executing finally clause


TypeError: unsupported operand type(s) for /: 'str' and 'str'

## 预定义的清理操作

某些对象定义了在不再需要该对象时要执行的标准清理操作
- 无论使用该对象的操作是成功还是失败，都会执行

In [21]:
# 执行完毕后，文件在一段不确定的时间内处于打开状态
for line in open("myfile.txt"):
    print(line, end="")

test!


In [22]:
# 执行完语句后，即使在处理行时遇到问题，文件 f 也始终会被关闭
with open("myfile.txt") as f:
    for line in f:
        print(line, end="")

test!
